In [2]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [69]:
cols = [7, 0, 1, 2, 3, 4, 5, 6, 10, 11, 12]

lookback = 30
horizon = 90
max_epochs = 300
batch_size = 1024
n_features =  len(cols)

split = [1,0.00001, 0.00001]
n_neurons=[25,25]
n_layers=2
dropout = 0.0
loss = 'mae'

lr = 0.0002

trainset = 'trainval'
testset = 'test'

In [88]:
trainval_df = pd.read_pickle('trainval_df.pkl')
X_trainval_sc = np.load('trainval_C_sc01.npy')
index_trainval = pd.read_pickle('trainval_C_index.pkl')

In [90]:
(trainval_df.shape, len(index_trainval), X_trainval_sc.shape)

((111388, 13), 111268, (111268, 30, 11))

In [74]:
def build_linear_LSTM(n_layers, nodes, horizon, lookback, n_features, lr, batch_size, return_sequences=False, dropout=False,
                      dropout_ratio=0.1, loss='mse', run_eagerly=False, stateful=False):
    """
    builds a linear LSTM with vector output through a dense layer
    """

    # Build
    model = Sequential()
    model.add(keras.Input(shape=(lookback, n_features), batch_size=batch_size))
    if n_layers > 1:
        for i in range(n_layers - 1):
            model.add(LSTM(nodes[i],
                           return_sequences=True,
                           stateful=stateful))
            if dropout is True:
                model.add(Dropout(dropout_ratio))

    model.add(LSTM(nodes[-1], return_sequences=return_sequences, stateful=stateful))
    if dropout is True:
        model.add(Dropout(dropout_ratio))

    # Vector output to predict multiple steps
    model.add(Dense(horizon, activation='linear'))  # Adjusted output layer

    # Compile
    opt = Adam(learning_rate=lr)
    model.compile(optimizer=opt, loss=loss, run_eagerly=run_eagerly)

    return model

In [75]:
model = build_linear_LSTM(n_layers=n_layers,
                          horizon=horizon,
                          lookback=lookback,
                          n_features=n_features,
                          nodes=n_neurons, 
                          lr=lr, 
                          batch_size=batch_size,
                          dropout=False, 
                          dropout_ratio=dropout, 
                          loss=loss,
                          stateful=False)


In [76]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                        │ (1024, 30, 25)              │           3,700 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (1024, 25)                  │           5,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (1024, 90)                  │           2,340 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,140 (43.52 KB)

 Trainable params: 11,140 (43.52 KB)

 Non-trainable params: 0 (0.00 B)